In [1]:
import numpy as np
import pandas as pd
import matplotlib as mt
import itertools as it
import Algorithm as al
import Support as sup
import random

In [2]:
ds=pd.read_csv("C:/Users/edoar/Desktop/UNI/Magi/MachineLearningandstat/Progetto_ML/your_dataset.csv")
X=ds.iloc[:,:-1]
Y=ds.iloc[:,-1]

In [3]:
print(ds.isnull().sum())
ds = ds.dropna()

x1     0
x2     0
x3     0
x4     0
x5     0
x6     0
x7     0
x8     0
x9     0
x10    0
y      0
dtype: int64


In [4]:
count1=np.sum(Y==1)
count_inv=np.sum(Y==-1)
print(f'{count1}, {count_inv}') #number of labels before removing outliers

X, Y = sup.Outliers(X, Y)
X.reset_index(drop=True, inplace=True)
Y.reset_index(drop=True, inplace=True)

count1=np.sum(Y==1)
count_inv=np.sum(Y==-1)
print(f'{count1}, {count_inv}') #number fo labels after removing outliers

4992, 5008
4571, 4885


In [5]:
np.random.seed(57)
random.seed(57)
X_train, X_test, Y_train, Y_test = sup.Split(X, Y)
X_train, X_test = sup.Standardized(X_train, X_test)

TUNING

In [6]:
perceptron_param = {
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

pegasos_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

logistic_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

In [7]:
np.random.seed(57)
random.seed(57)
loss_perc_in, param_perc_in = sup.tuning_par(al.Perceptron, pd.DataFrame(X_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos_in, param_pegasos_in = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit_in, param_logit_in = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc_in, loss_perc_in)
print(param_pegasos_in, loss_pegasos_in)
print(param_logit_in, loss_logit_in)

{'learning_rate': 0.001, 'n': 50} 0.33977527040289446
{'lambda_par': 0.1, 'n': 500} 0.2822044165568765
{'lambda_par': 0.1, 'n': 100} 0.5005156579922432


In [8]:
correlation_matrix = pd.DataFrame(X_train).corr()
high_corr = correlation_matrix[(correlation_matrix > 0.9) | (correlation_matrix < -0.9)]
high_corr = high_corr[high_corr != 1.0]
high_corr.stack().reset_index()

,level_0,level_1,0
0,2,5,-0.989999
1,2,9,-0.979744
2,5,2,-0.989999
3,5,9,0.989808
4,9,2,-0.979744
5,9,5,0.989808


In [9]:
np.random.seed(57)
random.seed(57)
perceptron_model=al.Perceptron(**param_perc_in)
pegasos_model=al.PegasosSVM(**param_pegasos_in)
logistic_model=al.RegLogisticClass(**param_logit_in)
perceptron_model.fit(X_train, Y_train)
pegasos_model.fit(X_train, Y_train)
logistic_model.fit(X_train, Y_train)

In [10]:
loss_perceptron = sup.Zero_One_Loss(Y_test, perceptron_model.predict(X_test))
loss_pegasos = sup.Zero_One_Loss(Y_test, pegasos_model.predict(X_test))
loss_logistic = sup.Zero_One_Loss(Y_test, logistic_model.predict(X_test))
print(f"0-1 Loss Perceptron : {loss_perceptron}  ")
print(f"0-1 Loss Pegasos : {loss_pegasos}  ")
print(f"0-1 Loss Logistic : {loss_logistic}  ")

0-1 Loss Perceptron : 0.3489601691928093  
0-1 Loss Pegasos : 0.3020796616143814  
0-1 Loss Logistic : 0.6792386323581248  


In [11]:
np.random.seed(57)
random.seed(57)
models=['Perceptron', 'Pegasos', 'Logistic']
loss_dicts = {model: {} for model in models}  # Create a dictionary for each model
for m in models:
    for i in [2,5,9,[5,9],[2,5],[2,9],[2,5,9]]:
        X_reduced_train=np.delete(X_train, i, axis=1)
        X_reduced_test=np.delete(X_test, i, axis=1)
        if m == 'Perceptron':
            model = al.Perceptron(**param_perc_in)
        elif m == 'Pegasos':
            model = al.PegasosSVM(**param_pegasos_in)
        else:
            model = al.RegLogisticClass(**param_logit_in)
        
        model.fit(X_reduced_train, Y_train)
        y_pred_test = model.predict(X_reduced_test)

        loss_test = sup.Zero_One_Loss(Y_test, y_pred_test)

        # Save test loss in the dictionary
        loss_dicts[m][tuple(i) if isinstance(i, list) else i] = loss_test

# Sort dictionaries by test loss in ascending order
sorted_loss_dicts = {
    model: dict(sorted(loss_dicts[model].items(), key=lambda item: item[1]))
    for model in models
}

for model in models:
    print(f"Sorted losses for {model}:")
    print(sorted_loss_dicts[model])
    print()

Sorted losses for Perceptron:
{(5, 9): np.float64(0.3193514275643285), (2, 9): np.float64(0.3239337328163553), (2, 5, 9): np.float64(0.33697567853366234), 2: np.float64(0.34966513923158266), 5: np.float64(0.35001762425096933), (2, 5): np.float64(0.39548819175185057), 9: np.float64(0.42051462812830454)}

Sorted losses for Pegasos:
{2: np.float64(0.3091293620021149), (2, 5): np.float64(0.3091293620021149), 9: np.float64(0.31018681706027496), (2, 5, 9): np.float64(0.3108917870990483), (2, 9): np.float64(0.312301727176595), (5, 9): np.float64(0.3168840324286218), 5: np.float64(0.31758900246739513)}

Sorted losses for Logistic:
{2: np.float64(0.4102925625660909), (5, 9): np.float64(0.4776172012689461), (2, 5): np.float64(0.5378921395840677), 5: np.float64(0.610504053577723), (2, 9): np.float64(0.6221360592174833), 9: np.float64(0.6263658794501233), (2, 5, 9): np.float64(0.6478674656327106)}



In [12]:
#To simplify our analysis we're gonna remove the same feature for all the models
X_train=np.delete(X_train, (5,9), axis=1)
X_test=np.delete(X_test, (5, 9), axis=1) 

DOING TUNING AGAIN WITHOUT THE FEATUREs AND SEEING IF THE BEST PARAMETERS CHANGE

In [13]:
np.random.seed(57)
random.seed(57)
loss_perc, param_perc = sup.tuning_par(al.Perceptron, pd.DataFrame(X_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos, param_pegasos = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit, param_logit = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc, loss_perc)
print(param_pegasos, loss_pegasos)
print(param_logit, loss_logit)

{'learning_rate': 1, 'n': 500} 0.3198037617458438
{'lambda_par': 1, 'n': 200} 0.28055383251852195
{'lambda_par': 0.1, 'n': 200} 0.538856735024245


In [14]:
np.random.seed(57)
random.seed(57)
perceptron_model=al.Perceptron(**param_perc)
pegasos_model=al.PegasosSVM(**param_pegasos)
logistic_model=al.RegLogisticClass(**param_logit)
perceptron_model.fit(X_train, Y_train)
pegasos_model.fit(X_train, Y_train)
logistic_model.fit(X_train, Y_train)
loss_perceptron = sup.Zero_One_Loss(Y_test, perceptron_model.predict(X_test))
loss_pegasos = sup.Zero_One_Loss(Y_test, pegasos_model.predict(X_test))
loss_logistic = sup.Zero_One_Loss(Y_test, logistic_model.predict(X_test))
print(f"0-1 Loss Perceptron : {loss_perceptron}  ")
print(f"0-1 Loss Pegasos : {loss_pegasos}  ")
print(f"0-1 Loss Logistic : {loss_logistic}  ")


0-1 Loss Perceptron : 0.33803313359182235  
0-1 Loss Pegasos : 0.3108917870990483  
0-1 Loss Logistic : 0.5805428269298555  


In [15]:
col_names = [f'x{i+1}' for i in range(X.shape[1])]
col_names=np.delete(col_names, (2,9))
sup.weights_comparison(perceptron_model.weights, pegasos_model.weights, logistic_model.weights, col_names)

  Feature  Perceptron   Pegasos  Logistic_Regression
0      x1    0.101221  0.102250             0.276443
1      x2    0.099007  0.037272             0.154135
2      x4    0.013334  0.157831             0.099160
3      x5    0.099952  0.087353             0.064245
4      x6    0.125197  0.032989             0.243959
5      x7    0.066880  0.213053             0.059493
6      x8    0.288993  0.218407             0.063198
7      x9    0.205416  0.150844             0.039367


POLYNOMIAL EXPANSION

In [16]:
X_poly_train, X_poly_test = sup.Polynomial_exp(X_train,X_test, col_names)
X_poly_train, X_poly_test = sup.Standardized(X_poly_train, X_poly_test)

In [17]:
correlation_matrix_poly = X_poly_train.corr()
high_corr_poly = correlation_matrix_poly[(correlation_matrix_poly > 0.9) | (correlation_matrix_poly < -0.9)]
high_corr_poly = high_corr_poly[high_corr_poly != 1.0]
high_corr_poly.stack().reset_index()

,level_0,level_1,0


In [18]:
np.random.seed(57)
random.seed(57)
loss_perc_poly, param_perc_poly = sup.tuning_par(al.Perceptron, pd.DataFrame(X_poly_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos_poly, param_pegasos_poly = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_poly_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit_poly, param_logit_poly = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_poly_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc_poly, loss_perc_poly)
print(param_pegasos_poly, loss_pegasos_poly)
print(param_logit_poly, loss_logit_poly)

{'learning_rate': 0.1, 'n': 200} 0.09004904817155868
{'lambda_par': 0.1, 'n': 500} 0.17707420906904167
{'lambda_par': 0.001, 'n': 200} 0.5333353838898031


In [19]:
np.random.seed(57)
random.seed(57)
perceptron_model_poly = al.Perceptron(**param_perc_poly)
pegasos_model_poly = al.PegasosSVM(**param_pegasos_poly)
logistic_model_poly = al.RegLogisticClass(**param_logit_poly)
perceptron_model_poly.fit(X_poly_train, Y_train)
pegasos_model_poly.fit(X_poly_train, Y_train)
logistic_model_poly.fit(X_poly_train, Y_train)

In [20]:
loss_perceptron_poly = sup.Zero_One_Loss(Y_test, perceptron_model_poly.predict(X_poly_test))
loss_pegasos_poly = sup.Zero_One_Loss(Y_test, pegasos_model_poly.predict(X_poly_test))
loss_logistic_poly = sup.Zero_One_Loss(Y_test, logistic_model_poly.predict(X_poly_test))
print(f"0-1 Loss Perceptron : {loss_perceptron_poly}")
print(f"0-1 Loss Pegasos : {loss_pegasos_poly}")
print(f"0-1 Loss Logistic : {loss_logistic_poly}")

0-1 Loss Perceptron : 0.10081071554458935
0-1 Loss Pegasos : 0.15509340853013748
0-1 Loss Logistic : 0.5904124074726824


In [21]:
sup.weights_comparison(perceptron_model_poly.weights, pegasos_model_poly.weights, logistic_model_poly.weights, X_poly_train.columns)

   Feature  Perceptron   Pegasos  Logistic_Regression
0       x1    0.029723  0.018755             0.012947
1       x2    0.012261  0.003791             0.019130
2       x4    0.019293  0.032400             0.012854
3       x5    0.013991  0.023269             0.068071
4       x6    0.050148  0.045630             0.011240
5       x7    0.030828  0.048297             0.033070
6       x8    0.130420  0.102653             0.034310
7       x9    0.045660  0.036568             0.040627
8     x1^2    0.025479  0.014881             0.005247
9     x2^2    0.013678  0.021167             0.017018
10    x4^2    0.001793  0.009556             0.016568
11    x5^2    0.009642  0.005888             0.006625
12    x6^2    0.000222  0.007354             0.026625
13    x7^2    0.002278  0.007043             0.064211
14    x8^2    0.023271  0.022661             0.019364
15    x9^2    0.019254  0.011414             0.004047
16   x1*x2    0.001791  0.045142             0.020167
17   x1*x4    0.010337  0.01

KERNEL

In [22]:
kernel_perc = al.KernelizedPerceptron('gaussian')
kernel_perc.fit(X_train, Y_train)
y_pred, weights = kernel_perc.predict(X_test)

In [23]:
kernel_perc_pol = al.KernelizedPerceptron('polynomial')
kernel_perc_pol.fit(X_train, Y_train)
y_pred_pol, weights_pol = kernel_perc_pol.predict(X_test)

In [24]:
loss_kernel_perceptron = sup.Zero_One_Loss(Y_test, y_pred)
loss_Kernel_poly = sup.Zero_One_Loss(Y_test, y_pred_pol)
print(f"0-1 Loss Kernelized-Perceptron Gaussian : {loss_kernel_perceptron}")
print(f"0-1 Loss Kernelized-Perceptron Polynomial : {loss_Kernel_poly}")

0-1 Loss Kernelized-Perceptron Gaussian : 0.062037363412054986
0-1 Loss Kernelized-Perceptron Polynomial : 0.09058864998237574


In [25]:
kernel_pegasos_gaussian = al.KernelizedPegasos('gaussian')
kernel_pegasos_gaussian.fit(X_train, Y_train)
y_pred_kernel_pegasos, weights_kernel_pegasos = kernel_pegasos_gaussian.predict(X_test)

In [26]:
kernel_pegasos_poly = al.KernelizedPegasos('polynomial')
kernel_pegasos_poly.fit(X_train, Y_train)
y_pred_kernel_pegasos_poly, weights_kernel_pegasos_poly = kernel_pegasos_poly.predict(X_test)

In [27]:
loss_kernel_pegasos = sup.Zero_One_Loss(Y_test, y_pred_kernel_pegasos)
loss_kernel_pegasos_poly = sup.Zero_One_Loss(Y_test, y_pred_kernel_pegasos_poly)
print(f"0-1 Loss Kernelized-Pegasos Gaussian:{loss_kernel_pegasos}")
print(f"0-1 Loss Kernelized-Pegasos Gaussian:{loss_kernel_pegasos_poly}")

0-1 Loss Kernelized-Pegasos Gaussian:0.212195981670779
0-1 Loss Kernelized-Pegasos Gaussian:0.2016214310891787
